In [ ]:
!pip install spacy_transformers
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 31.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 21.30 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
import json

cv_data = json.load(open(''))

In [ ]:
len(cv_data)

200

In [ ]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        ents = []
        entity_indices = set()  # Using a set for faster lookup
        for start, end, label in annot['entities']:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue
            entity_indices.update(range(start, end))
            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue
            if span is None:
                err_data = f"{[start, end]}  {text}\n"
                file.write(err_data)
            else:
                ents.append(span)
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass
    return db


In [ ]:
from sklearn.model_selection import train_test_split
train,test  = train_test_split(cv_data,test_size = 0.2)


In [ ]:
len(train),len(test)

(160, 40)

In [ ]:
file = open('error.txt','w')
db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()



/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 40/40 [00:00<00:00, 77.05it/s]


In [ ]:
!python -mspacy train  /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output  --paths.train ./train_data.spacy --paths.dev  ./test_data.spacy --gpu-id 0


ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline ================

In [ ]:
import spacy

# Load the trained model
nlp = spacy.load("en_core_web_sm")

# Save the model to disk
nlp.to_disk("/content/output/model-best")

In [ ]:
nlp =spacy.load("/content/output/model-best")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
nlp.to_disk("/content/output/model-best")

In [ ]:
import spacy

# Load the saved model from disk
nlp_loaded = spacy.load("/content/output/model-best")





/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
pip install PyMUPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 56.5 MB/s eta 0:00:00


In [ ]:
import sys,fitz

In [ ]:
fname = '/content/0be9c838-b957-7368-2bc7-00004d747bc2_resumeashvinideshmukh.pdf'
doc = fitz.open(fname)


In [ ]:
c

In [ ]:
text = text.strip()

In [ ]:
text

"Ashvini Deshmukh\nWeb Designer - Mahalasa Infotech\nMumbai, Maharashtra\nashavinideshmukh3_hpk@indeedemail.com - +919730743752\n• A result-oriented professional with Web Developer entrepreneurial experience in HTML, CSS, JavaScript etc.\n• Working with Mahalasa Infotech, Chembur-Mumbai as Web Designer.\n• Drove efforts towards maintaining Web development practices; adhered to company standards for coding.\n• Abilities in studying state-of-the-art development tools, programming techniques & computing equipment and\nmaintaining personal networks\n• Track record of providing solutions to client's requirements involving design and transforming according to\nbusiness rules.\n• A keen communicator with honed interpersonal, problem solving and negotiation abilities\nWilling to relocate: Anywhere\nEDUCATION\nB.Sc. in Computer Science\nSmt. K.W. College Sangli. secured -  Mumbai, Maharashtra\nADDITIONAL INFORMATION\nTECHNICAL SKILLS\nWeb Technologies: HTML5, CSS3, JavaScript, JQuery, Bootstrap

In [ ]:
doc = nlp_loaded(text)
for ent in doc.ents:
    print(ent.text, " ->>>>>>>>", ent.label_)

Ashvini Deshmukh  ->>>>>>>> Name
Web Designer  ->>>>>>>> Designation
Mahalasa Infotech
  ->>>>>>>> Companies worked at
Mumbai  ->>>>>>>> Location
+919730743752  ->>>>>>>> Email Address
Mahalasa Infotech  ->>>>>>>> Companies worked at
Smt. K.W. College Sangli. secured  ->>>>>>>> College Name


In [ ]:
import spacy
from google.colab import files

# Load the saved model from disk
model = spacy.load("/content/output/model-best")

# Save the spaCy model to a file using pickle
with open('spacy_model_2.pkl', 'wb') as f:
    pickle.dump(model,f)

files.download("spacy_model_2.pkl")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('spacy_model_2.pkl', 'rb') as f:
   mynlp = pickle.load(f)

In [ ]:
preprocessed_text

'ashvini deshmukh web designer  mahalasa infotech mumbai  maharashtra ashavinideshmukhhpk  indeedemailcom    resultoriented professional web developer entrepreneurial experience html  cs  javascript etc   working mahalasa infotech  chemburmumbai web designer   drove effort towards maintaining web development practice  adhered company standard coding   ability studying stateoftheart development tool  programming technique  computing equipment maintaining personal network  track record providing solution client requirement involving design transforming according business rule   keen communicator honed interpersonal  problem solving negotiation ability willing relocate  anywhere education bsc  computer science smt  kw  college sangli  secured  mumbai  maharashtra additional information technical skill web technology  html  cs  javascript  jquery  bootstrap  application  photoshop  dreamweaver etc  knowledge  word press  core php  mssql also knowledge c  c  vbnet  oracle '

In [ ]:
doc = mynlp(preprocessed_text)
for ent in doc.ents:
    print(ent.text, " ->>>>>>>>", ent.label_)

ashvini deshmukh  ->>>>>>>> Name
web designer  ->>>>>>>> Designation
mahalasa infotech mumbai  ->>>>>>>> College Name
maharashtra ashavinideshmukhhpk  ->>>>>>>> Location
mahalasa infotech  ->>>>>>>> Location
web designer  ->>>>>>>> Designation
computer science smt  ->>>>>>>> Degree
